In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from scipy import sparse
from sklearn.feature_extraction import FeatureHasher
from scipy.sparse import coo_matrix,csr_matrix,csc_matrix, hstack
from sklearn.preprocessing import normalize
from sklearn.utils import shuffle
from sklearn import linear_model
import gc
from sklearn import preprocessing

In [3]:
%ls

1_predata.ipynb                   submission_10_new.csv
3_prediction.ipynb                submission_11.csv
44fea_bst.model                   submission_11_new.csv
4_keras_nn.ipynb                  submission_nn_2.csv
5_random_forest.ipynb             submission_nn.csv
6_random_forest.ipynb             submission_nn_xgb
agencia_for_cliente_producto.csv  submission_xgb_10.pickle
agen_freq_semana.pickle           submission_xgb_2.csv
canal_for_cliente_producto.csv    submission_xgb.csv
clien_freq_semana.pickle          submission_xgb_nn_10.pickle
model_nn_10_after_l2reg.h5        submission_xgb_with_nn.csv
model_nn_10.h5                    train_pivot_3456_to_8.csv
model_nn_10_whole.h5              train_pivot_45678_to_9_new.csv
origin/                           train_pivot_56789_to_10_44fea.pickle
pivot_test.pickle                 train_pivot_56789_to_10_new.pickle
pivot_train_with_nan.pickle       train_pivot_56789_to_10.pickle
pivot_train_with_zero.pickle      train_pivot_6789_to_11_n

### begin training, for week 11
----------------

In [18]:
predictors_target_11 = ['LR_prod', 'LR_prod_corr',
       'NombreCliente',
       'agen_cliente_for_log_de', 'agen_for_log_de',
       'agen_producto_for_log_de', 'agen_ruta_for_log_de',
       'cliente_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_for_log_de', 'corr', 'pieces',
       'producto_for_log_de', 'ruta_cliente_for_log_de', 'ruta_for_log_de',
       'ruta_producto_for_log_de', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 't_m_2_cum', 't_m_3_cum',
       't_m_4_cum', 't_m_5_cum', 't_min_2', 't_min_3', 't_min_4',
       't_min_5', 'target', 'weight', 'weight_per_piece']

In [19]:
predictors_11 = ['LR_prod', 'LR_prod_corr',
       'NombreCliente',
       'agen_cliente_for_log_de', 'agen_for_log_de',
       'agen_producto_for_log_de', 'agen_ruta_for_log_de',
       'cliente_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_for_log_de', 'corr', 'pieces',
       'producto_for_log_de', 'ruta_cliente_for_log_de', 'ruta_for_log_de',
       'ruta_producto_for_log_de', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 't_m_2_cum', 't_m_3_cum',
       't_m_4_cum', 't_m_5_cum', 't_min_2', 't_min_3', 't_min_4',
       't_min_5', 'weight', 'weight_per_piece']

In [10]:
f = lambda x : (x-x.mean())/x.std(ddof=0)

In [14]:
train_pivot_xgb_time2 = pd.read_csv('train_pivot_xgb_time2.csv',index_col = 0)

In [7]:
train_pivot_6789_to_11 = pd.read_pickle('train_pivot_6789_to_11_new.pickle')

In [8]:
train_pivot_xgb_time2.head()

,Agencia_ID,Canal_ID,Cliente_ID,LR_prod,LR_prod_corr,NombreCliente,Producto_ID,Ruta_SAK,agen_cliente_for_log_de,agen_for_log_de,...,t_m_3_cum,t_m_4_cum,t_m_5_cum,t_min_2,t_min_3,t_min_4,t_min_5,target,weight,weight_per_piece
0,2061,2,26,2.001190,7.293554,18434,1182,7212,2.852285,3.491654,...,NaN,NaN,3.688879,NaN,NaN,NaN,3.688879,0.000000,210.0,210.00
1,2061,2,26,1.839411,6.703932,18434,4767,7212,2.852285,3.491654,...,NaN,NaN,3.761200,NaN,NaN,NaN,3.761200,3.761200,250.0,NaN
2,2061,2,26,1.911283,6.965878,18434,31393,7212,2.852285,3.491654,...,8.650325,5.877736,3.044522,2.772589,2.772589,2.833213,3.044522,3.135494,640.0,NaN
3,2061,2,26,3.113374,11.347029,18434,34204,7212,2.852285,3.491654,...,11.024839,7.218177,3.784190,3.555348,3.806662,3.433987,3.784190,3.828641,450.0,56.25
4,2061,2,26,2.031231,7.403043,18434,34206,7212,2.852285,3.491654,...,12.963710,9.202510,4.795791,4.248495,3.761200,4.406719,4.795791,4.499810,340.0,42.50


In [15]:
train_pivot_xgb_time2.columns.values

array(['Agencia_ID', 'Canal_ID', 'Cliente_ID', 'LR_prod', 'LR_prod_corr',
       'NombreCliente', 'Producto_ID', 'Ruta_SAK',
       'agen_cliente_for_log_de', 'agen_for_log_de',
       'agen_producto_for_log_de', 'agen_ruta_for_log_de',
       'cliente_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_for_log_de', 'corr', 'pieces',
       'producto_for_log_de', 'ruta_cliente_for_log_de', 'ruta_for_log_de',
       'ruta_producto_for_log_de', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 't_m_2_cum', 't_m_3_cum',
       't_m_4_cum', 't_m_5_cum', 't_min_2', 't_min_3', 't_min_4',
       't_min_5', 'target', 'weight', 'weight_per_piece'], dtype=object)

In [4]:
def normalize_dataset(train_dataset,test_dataset):
    train_dataset_normalize = train_dataset[predictors_11].copy()
    train_dataset_normalize['label'] = 0    
    
    test_dataset_normalize = test_dataset[predictors_11].copy()
    test_dataset_normalize['label'] = 1
    
    whole_dataset = pd.concat([train_dataset_normalize,test_dataset_normalize])
    whole_dataset_normalize = whole_dataset.apply(f,axis = 0)
    
    train_dataset_normalize = whole_dataset_normalize.loc[whole_dataset['label'] == 0]
    test_dataset_normalize = whole_dataset_normalize.loc[whole_dataset['label']==1]
    
    train_dataset_normalize.drop(['label'],axis = 1,inplace = True)
    test_dataset_normalize.drop(['label'],axis =1,inplace = True)
    
    train_dataset_normalize['target'] = train_dataset['target'].copy() 
    
#     target = train_dataset['target']
    return train_dataset_normalize,test_dataset_normalize

In [21]:
train_dataset_normalize, test_dataset_normalize = normalize_dataset(train_pivot_xgb_time2,train_pivot_6789_to_11)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
train_dataset_normalize.head()

Semana,LR_prod,LR_prod_corr,NombreCliente,agen_cliente_for_log_de,agen_for_log_de,agen_producto_for_log_de,agen_ruta_for_log_de,cliente_for_log_de,cliente_for_log_sum,cliente_producto_for_log_de,...,t_m_3_cum,t_m_4_cum,t_m_5_cum,t_min_2,t_min_3,t_min_4,t_min_5,weight,weight_per_piece,target
0,0.440004,0.007984,-1.198863,2.893915,7.040262,4.922515,3.707101,2.951726,0.023732,2.468511,...,NaN,NaN,0.841755,NaN,NaN,NaN,2.148223,0.041948,0.552880,0.000000
1,0.136858,0.004843,-1.198863,2.893915,7.040262,4.561988,3.707101,2.951726,0.023732,2.558317,...,NaN,NaN,0.888582,NaN,NaN,NaN,2.230611,0.180385,NaN,3.761200
2,0.271533,0.006239,-1.198863,2.893915,7.040262,3.000979,3.707101,2.951726,0.023732,1.433925,...,1.028141,0.851082,0.424536,1.090852,1.086921,1.149949,1.414172,1.530144,NaN,3.135494
3,2.524041,0.029578,-1.198863,2.893915,7.040262,3.917930,3.707101,2.951726,0.023732,2.414081,...,1.752516,1.424506,0.903467,1.968861,2.245187,1.825675,2.256800,0.872569,-0.290791,3.828641
4,0.496296,0.008568,-1.198863,2.893915,7.040262,5.719378,3.707101,2.951726,0.023732,3.231175,...,2.343992,2.273380,1.558475,2.746354,2.194265,2.919764,3.409216,0.491868,-0.366242,4.499810


In [26]:
train_pivot_xgb_time2_sample = train_dataset_normalize.sample(2000000)

train_feature_11 = train_pivot_xgb_time2_sample.drop(['target'],axis = 1)
train_label_11 = train_pivot_xgb_time2_sample[['target']]

dtrain_11 = xgb.DMatrix(train_feature_11,label = train_label_11,missing=np.nan)

In [27]:
num_round = 1000

cvresult = xgb.cv(param_11, dtrain_11, num_round, nfold=5,verbose_eval = 1,show_stdv=False,
                        seed = 0, early_stopping_rounds=5)
print(cvresult.tail())

[0]	train-rmse:1.14019	test-rmse:1.14021
[1]	train-rmse:0.961458	test-rmse:0.961473
[2]	train-rmse:0.824843	test-rmse:0.824873
[3]	train-rmse:0.722328	test-rmse:0.722397
[4]	train-rmse:0.647489	test-rmse:0.647603
[5]	train-rmse:0.592862	test-rmse:0.593041
[6]	train-rmse:0.555021	test-rmse:0.555254
[7]	train-rmse:0.528662	test-rmse:0.528924
[8]	train-rmse:0.510662	test-rmse:0.510977
[9]	train-rmse:0.498551	test-rmse:0.498903
[10]	train-rmse:0.49003	test-rmse:0.490423
[11]	train-rmse:0.484163	test-rmse:0.484588
[12]	train-rmse:0.480003	test-rmse:0.480457
[13]	train-rmse:0.477039	test-rmse:0.47752
[14]	train-rmse:0.474897	test-rmse:0.475405
[15]	train-rmse:0.473305	test-rmse:0.473844
[16]	train-rmse:0.472061	test-rmse:0.472641
[17]	train-rmse:0.471142	test-rmse:0.471752
[18]	train-rmse:0.470369	test-rmse:0.471
[19]	train-rmse:0.469729	test-rmse:0.470375
[20]	train-rmse:0.469123	test-rmse:0.469783
[21]	train-rmse:0.468683	test-rmse:0.469369
[22]	train-rmse:0.468239	test-rmse:0.468944
[23]	

In [24]:
param_11 = {'booster':'gbtree',
         'nthread': 10,
         'max_depth':5, 
         'eta':0.2,
         'silent':1,
         'subsample':0.7, 
         'objective':'reg:linear',
         'eval_metric':'rmse',
         'colsample_bytree':0.7}

In [28]:
num_round = 566

dtest_11 = xgb.DMatrix(test_dataset_normalize[predictors_11], missing=np.nan)
submission_11 = train_pivot_6789_to_11[['id']].copy()
j =0 
for j in range(20):
    
    train_pivot_xgb_time2_sample = train_dataset_normalize[predictors_target_11].sample(2000000)
    train_feature_11 = train_pivot_xgb_time2_sample.drop(['target'],axis = 1)
    train_label_11 = train_pivot_xgb_time2_sample[['target']]

    dtrain_11 = xgb.DMatrix(train_feature_11,label = train_label_11,missing= np.nan)
    
    bst_11 = xgb.train(param_11, dtrain_11, num_round)
    print str(j) + 'training finished!'
    submission_11['predict_' + str(j)] = bst_11.predict(dtest_11)

print 'finished'

0training finished!
1training finished!
2training finished!
3training finished!
4training finished!
5training finished!
6training finished!
7training finished!
8training finished!
9training finished!
10training finished!
11training finished!
12training finished!
13training finished!
14training finished!
15training finished!
16training finished!
17training finished!
18training finished!
19training finished!
finished


In [12]:
# make prediction
dtest_11 = xgb.DMatrix(train_pivot_6789_to_11[predictors], missing=NaN)
submission_11 = train_pivot_6789_to_11[['id']].copy()
submission_11['predict'] = bst.predict(dtest)
xgb.plot_importance(bst)

In [29]:
submission_11.to_csv('submission_11_new.csv')

In [11]:
submission_11 = pd.read_csv('submission_11_new.csv',index_col =0)

In [12]:
submission_11.columns.values

array(['id', 'predict_0', 'predict_1', 'predict_2', 'predict_3',
       'predict_4', 'predict_5', 'predict_6', 'predict_7', 'predict_8',
       'predict_9', 'predict_10', 'predict_11', 'predict_12', 'predict_13',
       'predict_14', 'predict_15', 'predict_16', 'predict_17',
       'predict_18', 'predict_19'], dtype=object)

### for week 10
-------

In [4]:
%ls

1_predata.ipynb                   submission_10_new.csv
3_prediction.ipynb                submission_11.csv
44fea_bst.model                   submission_11_new.csv
4_keras_nn.ipynb                  submission_nn_2.csv
5_random_forest.ipynb             submission_nn.csv
6_random_forest.ipynb             submission_nn_xgb
agencia_for_cliente_producto.csv  submission_xgb_10.pickle
agen_freq_semana.pickle           submission_xgb_2.csv
canal_for_cliente_producto.csv    submission_xgb.csv
clien_freq_semana.pickle          submission_xgb_nn_10.pickle
model_nn_10_after_l2reg.h5        submission_xgb_with_nn.csv
model_nn_10.h5                    train_pivot_3456_to_8.csv
model_nn_10_whole.h5              train_pivot_45678_to_9_new.csv
origin/                           train_pivot_56789_to_10_44fea.pickle
pivot_test.pickle                 train_pivot_56789_to_10_new.pickle
pivot_train_with_nan.pickle       train_pivot_56789_to_10.pickle
pivot_train_with_zero.pickle      train_pivot_6789_to_11_n

In [2]:
predictors_target_10 = ['ruta_freq', 'clien_freq', 'agen_freq', 'prod_freq',
       'agen_for_log_de', 'ruta_for_log_de', 'cliente_for_log_de',
       'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_agen_for_log_sum', 'corr', 't_min_1', 't_min_2',
       't_min_3', 't_min_4', 't_min_5', 't1_min_t2', 't1_min_t3',
       't1_min_t4', 't1_min_t5', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod', 'LR_prod_corr',
       'target', 't_m_5_cum', 't_m_4_cum', 't_m_3_cum', 't_m_2_cum',
       't_m_1_cum', 'NombreCliente', 'weight', 'weight_per_piece', 'pieces']

In [3]:
predictors_10 = ['ruta_freq', 'clien_freq', 'agen_freq', 'prod_freq',
       'agen_for_log_de', 'ruta_for_log_de', 'cliente_for_log_de',
       'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_agen_for_log_sum', 'corr', 't_min_1', 't_min_2',
       't_min_3', 't_min_4', 't_min_5', 't1_min_t2', 't1_min_t3',
       't1_min_t4', 't1_min_t5', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod', 'LR_prod_corr',
       't_m_5_cum', 't_m_4_cum', 't_m_3_cum', 't_m_2_cum',
       't_m_1_cum', 'NombreCliente', 'weight', 'weight_per_piece', 'pieces']

In [4]:
def normalize_dataset_10(train_dataset,test_dataset):
    train_dataset_normalize = train_dataset[predictors_10].copy()
    train_dataset_normalize['label'] = 0    
    
    test_dataset_normalize = test_dataset[predictors_10].copy()
    test_dataset_normalize['label'] = 1
    
    whole_dataset = pd.concat([train_dataset_normalize,test_dataset_normalize])
    whole_dataset_normalize = whole_dataset.apply(f,axis = 0)
    
    train_dataset_normalize = whole_dataset_normalize.loc[whole_dataset['label'] == 0]
    test_dataset_normalize = whole_dataset_normalize.loc[whole_dataset['label']==1]
    
    train_dataset_normalize.drop(['label'],axis = 1,inplace = True)
    test_dataset_normalize.drop(['label'],axis =1,inplace = True)
    
    train_dataset_normalize['target'] = train_dataset['target'].copy() 
    
#     target = train_dataset['target']
    return train_dataset_normalize,test_dataset_normalize

In [ ]:
dtypes = {'agen_for_log_de':'float32',
        'ruta_for_log_de':'float32',
        'cliente_for_log_de':'float32',
        'producto_for_log_de':'float32',
        'agen_ruta_for_log_de':'float32',
        'agen_cliente_for_log_de':'float32',
        'agen_producto_for_log_de':'float32',
        'ruta_cliente_for_log_de':'float32',
        'ruta_producto_for_log_de':"float32",
        'cliente_producto_for_log_de':'float32',
        'cliente_for_log_sum':'float32',
        'corr':'float32',
        't_min_1':'float32',
        't_min_2':'float32',
        't_min_3':'float32',
        't_min_4':'float32',
        't_min_5':'float32',
        't1_min_t2':'float32',
        't1_min_t3':'float32',
        't1_min_t4':'float32',
        't1_min_t5':'float32',
        't2_min_t3':'float32',
        't2_min_t4':'float32',
        't2_min_t5':'float32',
        't3_min_t4':'float32',
        't3_min_t5':'float32',
        't4_min_t5':'float32',
        'LR_prod':'float32',
        'LR_prod_corr':'float32',
        'target':'float32',
        't_m_5_cum':'float32',
        't_m_4_cum' :'float32',
        't_m_3_cum':'float32',
        't_m_2_cum':'float32',
        't_m_1_cum':'float32',
        'NombreCliente':'int32',
        'weight':'float32',
        'weight_per_piece':'float32',
        'pieces':'float32'}

In [ ]:
train_pivot_xgb_time1 = pd.read_csv('train_pivot_xgb_time1_44fea.csv',index_col = 0)

In [3]:
train_pivot_56789_to_10 = pd.read_pickle('train_pivot_56789_to_10_44fea.pickle')

In [4]:
train_pivot_56789_to_10.columns.values

array(['Cliente_ID', 'Producto_ID', 'id', 'sem10_sem11', 'Agencia_ID',
       'Canal_ID', 'Ruta_SAK', 'ruta_freq', 'clien_freq', 'agen_freq',
       'prod_freq', 'agen_for_log_de', 'ruta_for_log_de',
       'cliente_for_log_de', 'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_agen_for_log_sum', 'corr', 't_min_1', 't_min_2',
       't_min_3', 't_min_4', 't_min_5', 't1_min_t2', 't1_min_t3',
       't1_min_t4', 't1_min_t5', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod', 'LR_prod_corr',
       't_m_5_cum', 't_m_4_cum', 't_m_3_cum', 't_m_2_cum', 't_m_1_cum',
       'NombreCliente', 'weight', 'weight_per_piece', 'pieces'], dtype=object)

In [4]:
train_pivot_xgb_time1.columns.values

array(['Cliente_ID', 'Producto_ID', 'Agencia_ID', 'Canal_ID', 'Ruta_SAK',
       'ruta_freq', 'clien_freq', 'agen_freq', 'prod_freq',
       'agen_for_log_de', 'ruta_for_log_de', 'cliente_for_log_de',
       'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum',
       'cliente_producto_agen_for_log_sum', 'corr', 't_min_1', 't_min_2',
       't_min_3', 't_min_4', 't_min_5', 't1_min_t2', 't1_min_t3',
       't1_min_t4', 't1_min_t5', 't2_min_t3', 't2_min_t4', 't2_min_t5',
       't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod', 'LR_prod_corr',
       'target', 't_m_5_cum', 't_m_4_cum', 't_m_3_cum', 't_m_2_cum',
       't_m_1_cum', 'NombreCliente', 'weight', 'weight_per_piece', 'pieces'], dtype=object)

In [11]:
train_dataset_10_normalize, test_dataset_10_normalize = normalize_dataset_10(train_pivot_xgb_time1,
                                                                          train_pivot_56789_to_10)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [13]:
train_dataset_10_normalize.head()

Semana,agen_for_log_de,ruta_for_log_de,cliente_for_log_de,producto_for_log_de,agen_ruta_for_log_de,agen_cliente_for_log_de,agen_producto_for_log_de,ruta_cliente_for_log_de,ruta_producto_for_log_de,cliente_producto_for_log_de,...,t_m_5_cum,t_m_4_cum,t_m_3_cum,t_m_2_cum,t_m_1_cum,NombreCliente,weight,weight_per_piece,pieces,target
0,8.788259,5.529159,2.906576,3.130665,4.665460,3.354598,3.377005,3.292888,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.822053,1.391852,-0.188302,0.470006,4.574711
1,7.134035,5.691308,2.906576,-0.117290,3.590018,2.819381,3.083452,2.530998,2.647204,1.549154,...,1.173570,1.271504,1.338973,1.462613,1.794103,-1.180468,1.530289,NaN,NaN,2.639057
2,7.134035,5.691308,2.906576,2.136123,3.590018,2.819381,1.418874,2.530998,1.075756,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.180468,-0.061744,-0.435800,0.160219,2.397895
3,7.134035,5.691308,2.906576,2.236749,3.590018,2.819381,3.923257,2.530998,3.817979,2.509546,...,1.858372,2.182535,2.202313,2.287777,2.628881,-1.180468,0.872711,-0.291426,0.470006,3.784190
4,7.134035,5.691308,2.906576,0.250182,3.590018,2.819381,5.702605,2.530998,5.278630,3.337253,...,2.967155,2.699262,2.780972,2.979197,3.348328,-1.180468,0.492007,-0.367050,0.470006,4.682131


In [4]:
%ls

1_predata.ipynb                   stack_sub/
3_xgb.ipynb                       submission_10_new.csv
3_xgb_prediction.ipynb            submission_11_new.csv
4_keras_nn.ipynb                  submission_44fea.csv
5_random_forest.ipynb             submission_nn.csv
6_stack_model.ipynb               submission_nn_xgb
7_SGD_regressor.ipynb             test_dataset_10_normalize.csv
8_svm_linearSVR.ipynb             train_dataset_10_normalize.csv
agencia_for_cliente_producto.csv  train_dataset_10_normalize.pickle
canal_for_cliente_producto.csv    train_pivot_56789_to_10_44fea.pickle
old_submission/                   train_pivot_56789_to_10_new.pickle
origin/                           train_pivot_6789_to_11_new.pickle
pivot_test.pickle                 train_pivot_xgb_time1_44fea.csv
pivot_train_with_nan.pickle       train_pivot_xgb_time1.csv
RF_model/                         train_pivot_xgb_time2_38fea.csv
ruta_for_cliente_producto.csv     train_pivot_xgb_time2.csv


### read normalize_train
----------------------

In [6]:
train_dataset_10_normalize = pd.read_csv('train_dataset_10_normalize.csv',index_col = 0)
train_dataset_10_normalize.head()

,agen_for_log_de,ruta_for_log_de,cliente_for_log_de,producto_for_log_de,agen_ruta_for_log_de,agen_cliente_for_log_de,agen_producto_for_log_de,ruta_cliente_for_log_de,ruta_producto_for_log_de,cliente_producto_for_log_de,...,t_m_5_cum,t_m_4_cum,t_m_3_cum,t_m_2_cum,t_m_1_cum,NombreCliente,weight,weight_per_piece,pieces,target
0,8.788259,5.529158,2.906576,3.130665,4.665460,3.354598,3.377005,3.292888,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.822053,1.391852,-0.188302,0.470006,4.574711
1,7.134035,5.691308,2.906576,-0.117290,3.590019,2.819381,3.083452,2.530998,2.647204,1.549154,...,1.173570,1.271504,1.338973,1.462613,1.794103,-1.180468,1.530289,NaN,NaN,2.639057
2,7.134035,5.691308,2.906576,2.136122,3.590019,2.819381,1.418874,2.530998,1.075756,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.180468,-0.061744,-0.435800,0.160219,2.397895
3,7.134035,5.691308,2.906576,2.236749,3.590019,2.819381,3.923257,2.530998,3.817979,2.509546,...,1.858372,2.182535,2.202313,2.287777,2.628881,-1.180468,0.872711,-0.291426,0.470006,3.784190
4,7.134035,5.691308,2.906576,0.250182,3.590019,2.819381,5.702605,2.530998,5.278630,3.337253,...,2.967155,2.699262,2.780972,2.979197,3.348328,-1.180468,0.492007,-0.367050,0.470006,4.682131


In [5]:
test_dataset_10_normalize = pd.read_pickle('test_dataset_10_normalize_new.pickle')
test_dataset_10_normalize.head()

,agen_for_log_de,ruta_for_log_de,cliente_for_log_de,producto_for_log_de,agen_ruta_for_log_de,agen_cliente_for_log_de,agen_producto_for_log_de,ruta_cliente_for_log_de,ruta_producto_for_log_de,cliente_producto_for_log_de,...,t_m_5_cum,t_m_4_cum,t_m_3_cum,t_m_2_cum,t_m_1_cum,NombreCliente,weight,weight_per_piece,pieces,id
0,8.973844,6.397834,3.037557,1.835223,5.398326,3.786630,1.577787,3.812083,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.825542,1.706797,NaN,NaN,1569352
1,7.185299,5.729961,3.037557,5.701978,3.681476,2.891354,5.064185,2.595791,3.870621,2.473158,...,2.257555,2.204852,2.173276,2.286881,2.597253,-1.623858,1.876383,NaN,NaN,6667200
2,7.185299,5.729961,3.037557,4.924975,3.681476,2.891354,3.943551,2.595791,3.801173,1.670748,...,NaN,-0.273540,0.548709,0.768985,1.239590,-1.623858,0.976539,NaN,NaN,1592616
3,7.185299,5.729961,3.037557,6.049174,3.681476,2.891354,5.509614,2.595791,4.969614,3.183686,...,2.994623,2.686150,2.797207,2.862836,3.214847,-1.623858,1.668727,NaN,NaN,3909690
4,7.185299,5.729961,3.037557,3.371384,3.681476,2.891354,4.432333,2.595791,3.323528,2.531004,...,2.028818,2.125816,2.213651,2.354525,2.647532,-1.623858,1.277641,NaN,NaN,3659672


In [6]:
test_dataset_10_normalize.columns.values

array(['agen_for_log_de', 'ruta_for_log_de', 'cliente_for_log_de',
       'producto_for_log_de', 'agen_ruta_for_log_de',
       'agen_cliente_for_log_de', 'agen_producto_for_log_de',
       'ruta_cliente_for_log_de', 'ruta_producto_for_log_de',
       'cliente_producto_for_log_de', 'cliente_for_log_sum', 'corr',
       't_min_1', 't_min_2', 't_min_3', 't_min_4', 't_min_5', 't1_min_t2',
       't1_min_t3', 't1_min_t4', 't1_min_t5', 't2_min_t3', 't2_min_t4',
       't2_min_t5', 't3_min_t4', 't3_min_t5', 't4_min_t5', 'LR_prod',
       'LR_prod_corr', 't_m_5_cum', 't_m_4_cum', 't_m_3_cum', 't_m_2_cum',
       't_m_1_cum', 'NombreCliente', 'weight', 'weight_per_piece',
       'pieces', 'id'], dtype=object)

In [7]:
param_10 = {'booster':'gbtree',
         'nthread': 11,
         'max_depth':5, 
         'eta':0.1,
         'silent':1,
         'subsample':0.5, 
         'objective':'reg:linear',
         'eval_metric':'rmse',
         'colsample_bytree':0.7}

In [6]:
# train_dataset_10_normalize = train_pivot_xgb_time1[predictors_target_10].sample(2000000)

In [8]:
# train_pivot_xgb_time1_sample = train_dataset_10_normalize[predictors_target_10].sample(2000000)

train_feature_10 = train_dataset_10_normalize.drop(['target'],axis = 1)
train_label_10 = train_dataset_10_normalize[['target']]

dtrain_10 = xgb.DMatrix(train_feature_10,label = train_label_10,missing= np.nan)

In [9]:
gc.collect()

324

In [10]:
num_round = 20000

cvresult = xgb.cv(param_10, dtrain_10, num_round, nfold=5,show_stdv=False,
                        seed = 0, early_stopping_rounds=5,verbose_eval = 1)
print(cvresult.tail())

[0]	train-rmse:1.25273	test-rmse:1.25273
[1]	train-rmse:1.14826	test-rmse:1.14826
[2]	train-rmse:1.05531	test-rmse:1.05531
[3]	train-rmse:0.972803	test-rmse:0.972797
[4]	train-rmse:0.900266	test-rmse:0.900262
[5]	train-rmse:0.836907	test-rmse:0.836904
[6]	train-rmse:0.781313	test-rmse:0.781317
[7]	train-rmse:0.732719	test-rmse:0.732726
[8]	train-rmse:0.690846	test-rmse:0.690862
[9]	train-rmse:0.654807	test-rmse:0.654823
[10]	train-rmse:0.623818	test-rmse:0.623832
[11]	train-rmse:0.597409	test-rmse:0.597424
[12]	train-rmse:0.57508	test-rmse:0.575096
[13]	train-rmse:0.556027	test-rmse:0.556043
[14]	train-rmse:0.539989	test-rmse:0.540005
[15]	train-rmse:0.526533	test-rmse:0.526549
[16]	train-rmse:0.51534	test-rmse:0.515358
[17]	train-rmse:0.505916	test-rmse:0.505937
[18]	train-rmse:0.49811	test-rmse:0.498132
[19]	train-rmse:0.491551	test-rmse:0.491573
[20]	train-rmse:0.486143	test-rmse:0.486171
[21]	train-rmse:0.481592	test-rmse:0.481619
[22]	train-rmse:0.477847	test-rmse:0.477874
[23]	tr

In [11]:
num_round = 1900

bst = xgb.train(param_10, dtrain_10, num_round)

In [16]:
xgb.plot_importance(bst)

In [12]:
bst.save_model('bst_use_all_train.model')

In [ ]:
# dump model
bst= bst.dump_model('bst_use_all_train.model') 


In [4]:
bst = xgb.Booster({'nthread':10}) #init model
bst.load_model("bst_use_all_train.model") # load data

In [7]:
test_feature = test_dataset_10_normalize.drop(['id'],axis =1) 

In [8]:
dtest_10 = xgb.DMatrix(test_feature, missing=np.nan)

In [13]:
submission_10_all_train = pd.DataFrame()
submission_10_all_train = test_dataset_10_normalize[['id']].copy()

In [14]:
%time submission_10_all_train['predict'] = bst.predict(dtest_10)

CPU times: user 7min 57s, sys: 128 ms, total: 7min 57s
Wall time: 49.8 s


### begin combine for all train model
--------------------------------------

In [15]:
submission_10_all_train.head()

,id,predict
0,1569352,2.458774
1,6667200,3.593436
2,1592616,2.922797
3,3909690,4.209465
4,3659672,3.597292


In [11]:
submission_11 = pd.read_csv('submission_11_new.csv',index_col = 0)
submission_11.reset_index(inplace = True)

,predict
id,
1547831,4.406201
6825659,3.053817
5853787,2.684612
2316053,1.259826
900676,2.301486


In [18]:
submission_all_train = pd.concat([submission_10_all_train,submission_11])
submission_all_train['predict'] = submission_all_train['predict'].apply(np.expm1)
submission_all_train.head()

,id,predict
0,1569352,10.690468
1,6667200,35.358782
2,1592616,17.593215
3,3909690,66.320516
4,3659672,35.499250


In [19]:
submission_all_train['predict'] = submission_all_train['predict'].round(1) 

In [20]:
submission_all_train.rename(columns = {'predict':'Demanda_uni_equil'},inplace = True)

In [21]:
mask = submission_all_train[submission_all_train['Demanda_uni_equil'] <= 0].index
submission_all_train.loc[mask,'Demanda_uni_equil'] = 0
submission_all_train['Demanda_uni_equil'].describe()

count    6.999251e+06
mean     6.152023e+00
std      1.585226e+01
min      0.000000e+00
25%      1.900000e+00
50%      3.100000e+00
75%      5.700000e+00
max      2.645100e+03
Name: Demanda_uni_equil, dtype: float64

In [22]:
submission_all_train.to_csv('submission_all_train.csv',index = False)

In [23]:
submission_all_train.shape

(6999251, 2)

### begin cross validation
--------

In [21]:
num_round = 392

dtest_10 = xgb.DMatrix(test_dataset_10_normalize[predictors_10], missing=np.nan)
submission_10 = train_pivot_56789_to_10[['id']].copy()
i = 0

for i in range(20):
    train_pivot_xgb_time1_sample = train_dataset_10_normalize[predictors_target_10].sample(2000000)

    train_feature_10 = train_pivot_xgb_time1_sample.drop(['target'],axis = 1)
    train_label_10 = train_pivot_xgb_time1_sample[['target']]

    dtrain_10 = xgb.DMatrix(train_feature_10,label = train_label_10,missing= np.nan)
    
    bst = xgb.train(param_10, dtrain_10, num_round)
    print str(i) + 'training finished!'
    submission_10['predict_' + str(i)] = bst.predict(dtest_10)
    print str(i) + 'predicting finished!'


print 'finished'

0training finished!
0predicting finished!
1training finished!
1predicting finished!
2training finished!
2predicting finished!
3training finished!
3predicting finished!
4training finished!
4predicting finished!
5training finished!
5predicting finished!
6training finished!
6predicting finished!
7training finished!
7predicting finished!
8training finished!
8predicting finished!
9training finished!
9predicting finished!
10training finished!
10predicting finished!
11training finished!
11predicting finished!
12training finished!
12predicting finished!
13training finished!
13predicting finished!
14training finished!
14predicting finished!
15training finished!
15predicting finished!
16training finished!
16predicting finished!
17training finished!
17predicting finished!
18training finished!
18predicting finished!
19training finished!
19predicting finished!
finished


In [22]:
submission_10.to_csv('submission_10_new.csv')

In [26]:
# make prediction
xgb.plot_importance(bst)

### result for 44fea
---------------

In [13]:
submission_11['predict'] = submission_11[['predict_' + str(i) for i in range(20)]].mean(axis=1)

In [14]:
submission_11 = submission_11[['id','predict']]
gc.collect()
submission_11.head()

,id,predict
0,1547831,4.406201
1,6825659,3.053817
2,5853787,2.684612
3,2316053,1.259826
4,900676,2.301486


In [24]:
submission_11.to_csv('submission_11_new.csv',index = False)

In [16]:
submission_44fea = pd.concat([submission_44fea,submission_11],axis =0,copy = False)

In [17]:
submission_44fea.shape

(6999251, 2)

In [18]:
submission_44fea.rename(columns = {'predict':'Demanda_uni_equil'},inplace = True)
submission_44fea['Demanda_uni_equil'] = submission_44fea['Demanda_uni_equil'].apply(np.expm1)
submission_44fea.head()

Semana,id,Demanda_uni_equil
0,1569352,10.206472
1,6667200,35.766411
2,1592616,17.642273
3,3909690,62.235741
4,3659672,34.847991


In [19]:
submission_44fea['Demanda_uni_equil'] = submission_44fea['Demanda_uni_equil'].round(1)

In [20]:
submission_44fea['Demanda_uni_equil'].describe()

count    6.999251e+06
mean     6.075400e+00
std      1.606870e+01
min     -7.000000e-01
25%      1.900000e+00
50%      3.100000e+00
75%      5.600000e+00
max      2.879800e+03
Name: Demanda_uni_equil, dtype: float64

In [21]:
mask = submission_44fea[submission_44fea['Demanda_uni_equil'] <= 0].index
submission_44fea.loc[mask,'Demanda_uni_equil'] = 0
submission_44fea['Demanda_uni_equil'].describe()

count    6.999251e+06
mean     6.074749e+00
std      1.606832e+01
min      0.000000e+00
25%      1.900000e+00
50%      3.100000e+00
75%      5.600000e+00
max      2.879800e+03
Name: Demanda_uni_equil, dtype: float64

In [22]:
submission_44fea.head()

Semana,id,Demanda_uni_equil
0,1569352,10.2
1,6667200,35.8
2,1592616,17.6
3,3909690,62.2
4,3659672,34.8


In [23]:
submission_44fea.to_csv('submission_44fea.csv',index = False)

### combine week10 and week11
------------------

In [12]:
submission_10 = pd.read_csv('submission_10.csv',index_col = 0)

In [22]:
submission_10.shape

(3538385, 21)

In [23]:
submission_10.columns.values

array(['id', 'predict_0', 'predict_1', 'predict_2', 'predict_3',
       'predict_4', 'predict_5', 'predict_6', 'predict_7', 'predict_8',
       'predict_9', 'predict_10', 'predict_11', 'predict_12', 'predict_13',
       'predict_14', 'predict_15', 'predict_16', 'predict_17',
       'predict_18', 'predict_19'], dtype=object)

In [26]:
submission_11.columns.values

array(['id', 'predict_0', 'predict_1', 'predict_2', 'predict_3',
       'predict_4', 'predict_5', 'predict_6', 'predict_7', 'predict_8',
       'predict_9', 'predict_10', 'predict_11', 'predict_12', 'predict_13',
       'predict_14', 'predict_15', 'predict_16', 'predict_17',
       'predict_18', 'predict_19'], dtype=object)

In [27]:
submission = pd.concat([submission_10,submission_11],axis = 0)

In [28]:
submission.head()

Semana,id,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,...,predict_10,predict_11,predict_12,predict_13,predict_14,predict_15,predict_16,predict_17,predict_18,predict_19
0,1569352,1.570254,2.058795,2.034031,2.030173,2.360505,2.103804,2.306178,2.905277,2.163551,...,2.337900,2.052084,1.988659,2.145079,1.940696,1.745424,1.975164,1.969749,2.098094,2.157057
1,6667200,3.562964,3.605132,3.617031,3.654797,3.647879,3.615349,3.648028,3.547087,3.555520,...,3.617610,3.678215,3.664827,3.635223,3.621170,3.587815,3.619729,3.626218,3.718844,3.676515
2,1592616,3.095027,3.052559,2.981558,2.946887,2.818832,2.871925,2.925531,2.865346,2.925287,...,3.030976,3.111521,3.083862,2.991664,3.072153,2.873937,2.984680,2.915040,3.102967,3.057709
3,3909690,4.199105,4.092796,4.173037,4.114638,4.228617,4.108154,4.186193,4.186264,4.168976,...,4.217747,4.271296,4.210735,4.131866,4.077342,4.124441,4.189803,4.178765,4.237868,4.161020
4,3659672,3.582508,3.632398,3.602768,3.671225,3.653172,3.627649,3.626319,3.613807,3.573425,...,3.626101,3.640444,3.641191,3.594605,3.645134,3.609037,3.617180,3.677435,3.683105,3.669554


In [29]:
submission['predict'] = submission[['predict_' + str(i) for i in range(20)]].mean(axis=1)

In [30]:
submission.head()

Semana,id,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,...,predict_11,predict_12,predict_13,predict_14,predict_15,predict_16,predict_17,predict_18,predict_19,predict
0,1569352,1.570254,2.058795,2.034031,2.030173,2.360505,2.103804,2.306178,2.905277,2.163551,...,2.052084,1.988659,2.145079,1.940696,1.745424,1.975164,1.969749,2.098094,2.157057,2.122850
1,6667200,3.562964,3.605132,3.617031,3.654797,3.647879,3.615349,3.648028,3.547087,3.555520,...,3.678215,3.664827,3.635223,3.621170,3.587815,3.619729,3.626218,3.718844,3.676515,3.627489
2,1592616,3.095027,3.052559,2.981558,2.946887,2.818832,2.871925,2.925531,2.865346,2.925287,...,3.111521,3.083862,2.991664,3.072153,2.873937,2.984680,2.915040,3.102967,3.057709,2.989878
3,3909690,4.199105,4.092796,4.173037,4.114638,4.228617,4.108154,4.186193,4.186264,4.168976,...,4.271296,4.210735,4.131866,4.077342,4.124441,4.189803,4.178765,4.237868,4.161020,4.173270
4,3659672,3.582508,3.632398,3.602768,3.671225,3.653172,3.627649,3.626319,3.613807,3.573425,...,3.640444,3.641191,3.594605,3.645134,3.609037,3.617180,3.677435,3.683105,3.669554,3.633988


In [32]:
submission.rename(columns = {'predict':'Demanda_uni_equil'},inplace = True)
submission['Demanda_uni_equil'] = submission['Demanda_uni_equil'].apply(np.expm1)
submission.head()

Semana,id,predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,...,predict_11,predict_12,predict_13,predict_14,predict_15,predict_16,predict_17,predict_18,predict_19,Demanda_uni_equil
0,1569352,1.570254,2.058795,2.034031,2.030173,2.360505,2.103804,2.306178,2.905277,2.163551,...,2.052084,1.988659,2.145079,1.940696,1.745424,1.975164,1.969749,2.098094,2.157057,7.354917
1,6667200,3.562964,3.605132,3.617031,3.654797,3.647879,3.615349,3.648028,3.547087,3.555520,...,3.678215,3.664827,3.635223,3.621170,3.587815,3.619729,3.626218,3.718844,3.676515,36.618239
2,1592616,3.095027,3.052559,2.981558,2.946887,2.818832,2.871925,2.925531,2.865346,2.925287,...,3.111521,3.083862,2.991664,3.072153,2.873937,2.984680,2.915040,3.102967,3.057709,18.883251
3,3909690,4.199105,4.092796,4.173037,4.114638,4.228617,4.108154,4.186193,4.186264,4.168976,...,4.271296,4.210735,4.131866,4.077342,4.124441,4.189803,4.178765,4.237868,4.161020,63.927416
4,3659672,3.582508,3.632398,3.602768,3.671225,3.653172,3.627649,3.626319,3.613807,3.573425,...,3.640444,3.641191,3.594605,3.645134,3.609037,3.617180,3.677435,3.683105,3.669554,36.863517


In [33]:
submission_final = submission[['id','Demanda_uni_equil']].copy()

In [34]:
submission_final['Demanda_uni_equil'] = submission_final['Demanda_uni_equil'].round(1)

In [35]:
submission_final.head()

Semana,id,Demanda_uni_equil
0,1569352,7.4
1,6667200,36.6
2,1592616,18.9
3,3909690,63.9
4,3659672,36.9


In [36]:
submission_final.to_csv('submission_xgb_2.csv',index = False)